#### I. Referencias consultadas

# Numerical development to resolve Time-Dependent Schrödinger equation through eigenvalue problem

## 1. Import modules with useful functions, parameters, and algorithms.

In [428]:
name_code = "03_Code_auxiliary_tests";

In [429]:
include("../modules/module_schrodinger_equation_eigenproblem.jl"); # módulo para construir grilla (1D)

  Activating project at `~/github_repositories/my_repositories/ElectronNuclear_Coupling_Dynamics/gridap_makie`


## 2. Setting grid properties

* Tipo de grilla
* Discretización espacial
* Dominio
* Condiciones de contorno

In [430]:
Bohr_radius_meter=5.29177210903e−11;                        # [m]
Angstrom_to_meter=1e−10;                                    # [m/Å]
Angstrom_to_au=Angstrom_to_meter*(1.0/Bohr_radius_meter);   # [au/Å]

## 5. Resolve the 2D problem

In [431]:
function space_coord_2D(dom,Δx,Δy)
    nx=round(Int,1.0/Δx)+1; # cantidad de puntos en dirección x
    ny=round(Int,1.0/Δy)+1; # cantidad de puntos en dirección y
    x=[dom[1]+abs(dom[2]-dom[1])*Δx*(i-1) for i in 1:nx];
    y=[dom[3]+abs(dom[4]-dom[3])*Δy*(i-1) for i in 1:ny];
    pts=[Point(x[i],y[j]) for i in 1:nx for j in 1:ny];
    return x,y,pts;
end

space_coord_2D (generic function with 1 method)

In [432]:
dom_2D=(-12.0*Angstrom_to_au,12.0*Angstrom_to_au,-4.9*Angstrom_to_au,4.9*Angstrom_to_au);   # cantidad de FE y dominio espacial
n_1D_r=49;n_1D_R=49;    # cantidad de FE por dimension (cantidad de intervalos)
ΔrH=1.0/n_1D_r; ΔRH=1.0/n_1D_R                        # tamaño del elemento 2D
partition_2D=(n_1D_r,n_1D_R);                               # grilla de tamaño n²
model_2D=CartesianDiscreteModel(dom_2D,partition_2D);   # creamos modelo con elementos cartesianos

In [433]:
writevtk(model_2D,path_models*"CartesianDiscreteModel");

In [434]:
dirichlet_values_2D=(0.0+im*0.0);        # condiciones de contorno de tipo fulldirichlet
dirichlet_tags_2D="boundary";

In [435]:
Ω_2D,dΩ_2D,Γ_2D,dΓ_2D=measures(model_2D,3,dirichlet_tags_2D);
reffe_2D=ReferenceFE(lagrangian,Float64,2);
DOF_r,DOF_R,pts=space_coord_2D(dom_2D,ΔrH,ΔRH);

In [436]:
VHre_2D=TestFESpace(model_2D,reffe_2D;vector_type=Vector{ComplexF64},conformity=:H1,dirichlet_tags=dirichlet_tags_2D);
UHre_2D=TrialFESpace(VHre_2D,dirichlet_values_2D);

R₁=-5.0*Angstrom_to_au;R₂=5.0*Angstrom_to_au;Rc=1.5*Angstrom_to_au;Rf=1.5*Angstrom_to_au; # parameters

pH_2D,qH_2D,rH_2D=eigenvalue_problem_functions((R₁,R₂,Rc,Rf);switch_potential = "Electron_Nuclear_Potential_2D")
aH_2D,bH_2D=bilineal_forms(pH_2D,qH_2D,rH_2D,dΩ_2D);

Set Electron-Nuclear potential


In [437]:
nevH=4;
probH_2D=EigenProblem(aH_2D,bH_2D,UHre_2D,VHre_2D;nev=nevH,tol=10^(-9),maxiter=300,explicittransform=:none,sigma=-1.0);

In [438]:
ϵH_2D,ϕH_2D=solve(probH_2D);

In [439]:
# Escribimos autovectores para visualizar externamente
write_data=true;
if write_data
    for i in 1:nevH
        writevtk(Ω_2D,path_images*"eigenprob_2D_01_num$(i)",cellfields=["phi_real" => real(ϕH_2D[i]), "phi_imag" => imag(ϕH_2D[i])]);
    end
end

In [440]:
S_2D=TimeIndependet_Diff_Shannon_Entropy_1D(ϕH_2D,UHre_2D,dΩ_2D)

4-element Vector{Float64}:
 5.229508859534801
 5.071587361002997
 5.033956654014375
 5.277484096650664

In [441]:
𝛹ₓᵢ=interpolate_everywhere(ϕH_2D[1],UHre_2D);
𝛹ₓᵢ=𝛹ₓᵢ/norm_L2(𝛹ₓᵢ,dΩ_2D);
ρₓᵢ=real(𝛹ₓᵢ'*𝛹ₓᵢ);

In [547]:
include("../modules/module_schrodinger_equation_eigenproblem.jl"); # módulo para construir grilla (1D)

  Activating project at `~/github_repositories/my_repositories/ElectronNuclear_Coupling_Dynamics/gridap_makie`


In [548]:
δKroneckerGridap = CellField(x->kronecker_deltax_Gridap_v2(x,DOF_r[1],1.0),Ω_2D);
δnorm=sum(integrate(δKroneckerGridap,dΩ_2D));
δKroneckerGridap = CellField(x->kronecker_deltax_Gridap_v2(x,DOF_r[1],δnorm),Ω_2D);

In [549]:
writevtk(Ω_2D,path_images*"aux_function",cellfields=["δKroneckerGridap" =>δKroneckerGridap]);

In [550]:
sum(integrate(δKroneckerGridap,dΩ_2D))

1.0

In [551]:
-sum(integrate(ρₓᵢ*(log∘ρₓᵢ)*δKroneckerGridap,dΩ_2D))

2.0739996029051654e-11

In [ ]:
function trapez1D_partial_integ_function2D(x_vec,y_value,function2D)
    coef_vec=copy(x_vec);
    function_vec=copy(x_vec);
    dim_x=length(x_vec);

    coef_vec[1]=1.0;
    coef_vec[2:(dim_x-1)].=2.0;
    coef_vec[end]=1.0;

    function_vec[1]=function2D(x_vec[1],y_value);
    function_vec[end]=function2D(x_vec[end],y_value);

    for i in 2:(length(x_vec)-1);
        function_vec[i]=function2D(x_vec[i-1],y_value);
    end

    return 0.5*Δx*(transpose(function_vec)*coef_vec);
end

trapez1D_partial_integ_function2D (generic function with 2 methods)

In [ ]:
function function2D(x_value,y_value)
    return cos(x_value*y_value);
end

function2D (generic function with 1 method)

In [ ]:
trapez1D_partial_integ_function2D(DOF_r,ΔrH,DOF_R[1],function2D)

-0.032508965091200936

In [ ]:
(sin(-DOF_r[1]*DOF_R[1])+sin(DOF_r[end]*DOF_R[1]))/DOF_R[1]

0.10508264367833813

In [ ]:
function trapez2D_integ(x_vec,y_vec,Δx,Δy,function2D)

    coef_vec=copy(y_vec);
    function_vec=copy(y_vec);
    dim_y=length(y_vec);

    coef_vec[1]=y_vec[1];
    coef_vec[2:(dim_y-1)].=2.0;
    coef_vec[end]=y_vec[end];

    function_vec[1]=function2D(coef_vec[1],y_value);
    function_vec[1]=trapez1D_partial_integ_function2D(x_vec,Δx,y_value,function2D)


    function_vec[end]=function2D(coef_vec[end],y_value);

    for i in 2:(length(x_vec)-1);
        function_vec[i]=function2D(x_vec[i-1],y_value);
    end

    return 0.5*Δx*(transpose(function_vec)*coef_vec);
end

trapez2D_integ (generic function with 1 method)